In [12]:
import pandas as pd
import os
import requests
from PIL import Image
from io import BytesIO
import random

### جمع آوری داده و لیبل زدن 

In [13]:
# ثابت کردن seed برای تکرارپذیری
random.seed(42)

# فرض می‌کنیم دیتا از دیتابیس به صورت یک فایل CSV استخراج شده و در قالب DataFrame pandas داریم
data = pd.read_csv("BaSalam.products.csv")

# شمارش تعداد تکرار هر دسته
category_counts = data['new_categoryId'].value_counts()

# انتخاب ۵۰ دسته پر تکرار
top_categories = category_counts.iloc[0:51].index

# فیلتر کردن دیتا برای فقط این ۵۰ دسته
filtered_data = data[data['new_categoryId'].isin(top_categories)]

C:\Users\Latitude 5289 2-in-1\AppData\Local\Temp\ipykernel_25144\3103698580.py:5: DtypeWarning: Columns (18,35,36,37,38,42) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("BaSalam.products.csv")


In [14]:
code_to_title=filtered_data[["new_categoryId","categoryTitle"]].drop_duplicates()

In [15]:
code_to_title

,new_categoryId,categoryTitle
3,133.0,ادویه
6215,248.0,مانتو و تونیک
6216,260.0,کفش، دمپایی مردانه
6218,242.0,کفش و دمپایی زنانه
6224,247.0,لباس زیر زنانه
6228,228.0,اکسسوری زنانه
6265,238.0,شال و روسری زنانه
6273,235.0,زیورآلات زنانه
11257,324.0,فلاسک، کلمن و ماگ
11258,312.0,بذر و تخم گیاهان


In [3]:
filtered_data=filtered_data[filtered_data['new_categoryId']!=790.0]

In [19]:
filtered_data.groupby(['new_categoryId'])["_id"].count()

new_categoryId
133.0     9270
228.0     8740
235.0     8967
238.0     8695
242.0     8863
247.0     8813
248.0     8687
260.0     8926
305.0     8909
314.0     9169
359.0     8978
434.0     9056
435.0     9018
474.0     9054
512.0     8685
569.0     8985
732.0     8869
782.0     9132
903.0     8891
1003.0    8758
Name: _id, dtype: int64

In [ ]:
# تابعی برای دانلود و تغییر سایز عکس‌ها
#دانلود و تغیر سایز و فرمت  
def download_and_resize_image(url, save_path, size=(128, 128), max_retries=1):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                img = img.resize(size)  # تغییر سایز عکس
                            # تبدیل به حالت RGB (در صورتی که تصویر RGBA باشد)
                if img.mode == 'RGBA':
                    img = img.convert('RGB')
                
                # ذخیره تصویر به فرمت JPEG
                img.save(save_path, 'JPEG')
                return True
            else:
                print(f"Failed to download image from {url}. Status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Attempt {retries + 1} failed for {url}: {e}")
            #time.sleep(1)  # تاخیر قبل از تلاش مجدد
        retries += 1
    return False

In [ ]:
# تابعی برای جایگزینی عکس فیل شده
#در صورت فیل شدن تا 5 بار عکسی دیگر را تلاش میکنه دانلود کنه
def download_image_with_retry(category_data, save_path, size=(128, 128), max_retries=5):
    retries = 0
    while retries < max_retries:
        # انتخاب تصادفی از محصولاتی که هنوز دانلود نشده‌اند
        product_row = category_data.sample(n=1, random_state=random.randint(0, 1000))
        #photo_urls = [product_row['photo_MEDIUM'].values[0], product_row['photo_SMALL'].values[0]]  # لیست URLهای عکس
        if download_and_resize_image(product_row['photo_MEDIUM'].values[0], save_path, size):
            print("ok")
            return True
        retries += 1
        print(f"Retrying... ({retries}/{max_retries})")
        #time.sleep(1)  # اضافه کردن تاخیر برای جلوگیری از درخواست‌های مکرر
    return False  # اگر بعد از تلاش‌های متعدد ناموفق بود، False برمی‌گردانیم


In [8]:
top_categories[1:]

Index([385.0, 791.0], dtype='float64', name='new_categoryId')

In [ ]:
# دانلود و تغییر سایز عکس‌ها برای هر دسته
for category_id in top_categories:
    category_data = filtered_data[filtered_data['new_categoryId'] == category_id]
    
    # انتخاب رندوم 1200 نمونه از این دسته
    sampled_data = category_data.sample(n=1200 ,random_state=42)  # با seed ثابت برای انتخاب تصادفی
    
    # ایجاد فولدر برای دسته
    os.makedirs(f"images_2/{int(category_id)}", exist_ok=True)
    
    for i, row in sampled_data.iterrows():
        save_path = f"images_2/{int(category_id)}/{row['_id']}.jpg"
        
        # دانلود و تغییر سایز عکس
        if not download_and_resize_image(row['photo_MEDIUM'], save_path):
            print(f"First attempt failed, retrying for product ID {row['_id']}")
            if not download_image_with_retry(category_data, save_path):
                print(f"Failed to download image for product ID {row['_id']} after multiple retries")

Failed to download image from https://statics.basalam.com/public-6/users/KEd1D/01-11/lELsOuDkJ18mgsrlVLY4VcqeZKaTSlETrNdz2UhiEjsM21Zt8L.jpg_512X512X70.jpg. Status code: 403
First attempt failed, retrying for product ID 5863775
ok
Failed to download image from https://statics.basalam.com/public-6/users/0JXL/2107/U99x0kPrO0Up5BcNn2eUfClL5kgVAjmG9cuJ9ctA.jpeg_512X512X70.jpeg. Status code: 403
First attempt failed, retrying for product ID 2200364
ok
Failed to download image from https://statics.basalam.com/public-14/users/KEd1D/10-23/XBVoQpRiULFcJEz9h8STkJEU00hEXEe5yxghkUAXfvZlqQSiba.jpg_512X512X70.jpg. Status code: 403
First attempt failed, retrying for product ID 3129580
ok
Failed to download image from https://statics.basalam.com/public-7/users/PaNVk/01-17/vKQbS95jMW3o7dVMdkfQzJJP1w5BBSZ4QOQaNfe0id7Jk4104b.jpg_512X512X70.jpg. Status code: 403
First attempt failed, retrying for product ID 3207957
ok
Failed to download image from https://statics.basalam.com/public-10/users/oDPdM/07-26/uua

### شروع آموزش

In [2]:
!pip install torch

  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB 445.2 kB/s eta 0:07:37
   ---------------------------------------- 0.0/203.1 MB 445.2 kB/s eta 0:07:37
   ---------------------------------------- 0.0/203.1 MB 219.4 kB/s eta 0:15:26
   ---------------------------------------- 0.0/203.1 MB 219.4 kB/s eta 0:15:26
   ---------------------------------------- 0.0/203.1 MB 219

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    unknown package:
        Expected sha256 603c52d2fe06433c18b747d25f5c333f9c1d58615620578c326d66f258686f9a
             Got        0c08c80097ea6852c2b51eb823a1c0249d4e2c7ec9fad7e397e41c12822d6f96



   - -------------------------------------- 7.1/203.1 MB 68.4 kB/s eta 0:47:46
   - -------------------------------------- 7.1/203.1 MB 68.9 kB/s eta 0:47:23
   - -------------------------------------- 7.1/203.1 MB 68.9 kB/s eta 0:47:23
   - -------------------------------------- 7.1/203.1 MB 68.9 kB/s eta 0:47:23
   - -------------------------------------- 7.1/203.1 MB 68.9 kB/s eta 0:47:23
   - -------------------------------------- 7.1/203.1 MB 68.9 kB/s eta 0:47:23
   - -------------------------------------- 7.2/203.1 MB 69.0 kB/s eta 0:47:19
   - -------------------------------------- 7.2/203.1 MB 69.0 kB/s eta 0:47:19
   - -------------------------------------- 7.2/203.1 MB 69.0 kB/s eta 0:47:19
   - -------------------------------------- 7.2/203.1 MB 69.0 kB/s eta 0:47:19
   - -------------------------------------- 7.2/203.1 MB 69.0 kB/s eta 0:47:19
   - -------------------------------------- 7.2/203.1 MB 68.5 kB/s eta 0:47:42
   - -------------------------------------- 7.2/203.

In [2]:
!pip install torchvision

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 1.6/1.6 MB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 203.1/203.1 MB 3.9 MB/s eta 0:00:00
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Attempting uninstall: torch
    Found existing installation: torch 2.1.1
    Uninstalling torch-2.1.1:
      Successfully uninstalled torch-2.1.1


  You can safely remove it manually.

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score

# تعریف تنظیمات و پارامترها
IMAGE_SIZE = 128
BATCH_SIZE = 24
EPOCHS = 30
LEARNING_RATE = 0.0001


In [2]:

# 1. آماده‌سازی داده‌ها
class CustomDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.categories = os.listdir(image_dir)
        
        # خواندن تصاویر و برچسب‌ها
        for label, category in enumerate(self.categories):
            category_path = os.path.join(image_dir, category)
            for image_name in os.listdir(category_path):
                if image_name.endswith(".jpg") or image_name.endswith(".png"):
                    self.images.append(os.path.join(category_path, image_name))
                    self.labels.append(label)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert("RGB")
        
        if self.transform:
            img = self.transform(img)
        
        return img, label

# تعریف transform‌ها برای داده‌ها
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor()
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# بارگذاری داده‌ها
image_dir = "E:\diginext\proj\good_images"
dataset = CustomDataset(image_dir, transform)
train_data, test_data = train_test_split(dataset, test_size=0.2, stratify=dataset.labels)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# # هنگام تقسیم داده‌ها، تصاویر و برچسب‌ها را ذخیره کنید
# train_indices, test_indices = train_test_split(
#     list(range(len(dataset))),
#     test_size=0.2,
#     stratify=dataset.labels,
# )

# train_data = [dataset[i] for i in train_indices]
# test_data = [dataset[i] for i in test_indices]
# test_image_paths = [dataset.images[i] for i in test_indices]  # ذخیره مسیر تصاویر تست

# # لودرهای داده
# train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

#### مدل رگرسیون خطی

In [27]:

# 2. مدل رگرسیون لاجستیک
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        x = self.flatten(x)
        return self.fc(x)

# محاسبه سایز ورودی
sample_image, _ = dataset[0]
input_size = np.prod(sample_image.shape)

# ساخت مدل رگرسیون لاجستیک
logistic_model = LogisticRegressionModel(input_size, len(dataset.categories))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(logistic_model.parameters(), lr=LEARNING_RATE)

# آموزش مدل رگرسیون لاجستیک
def train_logistic_model(model, train_loader, criterion, optimizer, epochs=EPOCHS):
    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
        
train_logistic_model(logistic_model, train_loader, criterion, optimizer)

# ارزیابی مدل رگرسیون لاجستیک
def evaluate_logistic_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Logistic Regression Accuracy: {accuracy * 100:.2f}%")
    
evaluate_logistic_model(logistic_model, test_loader)


Epoch 1/30, Loss: 7.121187210083008
Epoch 2/30, Loss: 4.430386543273926
Epoch 3/30, Loss: 4.050364017486572
Epoch 4/30, Loss: 5.628933429718018
Epoch 5/30, Loss: 11.863256454467773
Epoch 6/30, Loss: 5.505681991577148
Epoch 7/30, Loss: 3.4939494132995605
Epoch 8/30, Loss: 4.7863945960998535
Epoch 9/30, Loss: 6.046133995056152
Epoch 10/30, Loss: 2.21878719329834
Epoch 11/30, Loss: 6.674868106842041
Epoch 12/30, Loss: 11.267661094665527
Epoch 13/30, Loss: 7.9444193840026855
Epoch 14/30, Loss: 2.615816354751587
Epoch 15/30, Loss: 4.803086757659912
Epoch 16/30, Loss: 3.1449437141418457
Epoch 17/30, Loss: 4.421376705169678
Epoch 18/30, Loss: 2.9755356311798096
Epoch 19/30, Loss: 4.845274925231934
Epoch 20/30, Loss: 6.349038124084473
Epoch 21/30, Loss: 2.721377372741699
Epoch 22/30, Loss: 4.6249003410339355
Epoch 23/30, Loss: 8.377314567565918
Epoch 24/30, Loss: 8.605525970458984
Epoch 25/30, Loss: 3.10941743850708
Epoch 26/30, Loss: 3.073237895965576
Epoch 27/30, Loss: 5.000927448272705
Epoc

In [3]:

# 3. مدل EfficientNet-B0 (با استفاده از Transfer Learning)
efficientnet_model = models.efficientnet_b0(pretrained=True)
for param in efficientnet_model.parameters():
    param.requires_grad = False  # فریز کردن لایه‌های ابتدایی

# باز کردن دو لایه آخر برای یادگیری
for param in list(efficientnet_model.classifier[-2:].parameters()):
    param.requires_grad = True
    
# تنظیم لایه‌های نهایی
num_features = efficientnet_model.classifier[1].in_features
efficientnet_model.classifier[1] = nn.Linear(num_features, len(dataset.categories))

# آموزش مدل EfficientNet-B0
efficientnet_model = efficientnet_model.to("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet_model.parameters(), lr=LEARNING_RATE)

# آموزش مدل EfficientNet-B0
def train_efficientnet_model(model, train_loader, criterion, optimizer, epochs=EPOCHS):
    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
        
train_efficientnet_model(efficientnet_model, train_loader, criterion, optimizer)

# ارزیابی مدل EfficientNet-B0
def evaluate_efficientnet_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"EfficientNet-B0 Accuracy: {accuracy * 100:.2f}%")
    
evaluate_efficientnet_model(efficientnet_model, test_loader)


c:\Users\Latitude 5289 2-in-1\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Latitude 5289 2-in-1\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [55]:
from sklearn.metrics import classification_report

# ارزیابی مدل EfficientNet-B0 با نمایش دقت هر دسته
def evaluate_efficientnet_model_per_category(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # نمایش دقت کلی
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Overall Accuracy: {accuracy * 100:.2f}%")

    # نمایش دقت هر دسته با استفاده از classification_report
    category_names = dataset.categories  # نام دسته‌ها از داده‌ها
    report = classification_report(all_labels, all_preds, target_names=category_names)
    print("\nCategory-wise Accuracy:\n", report)

# فراخوانی تابع ارزیابی
evaluate_efficientnet_model_per_category(efficientnet_model, test_loader)


ValueError: too many values to unpack (expected 2)

In [4]:
# اصلاح کلاس CustomDataset برای حفظ مسیرها
class CustomDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.categories = os.listdir(image_dir)
        
        # خواندن تصاویر و برچسب‌ها
        for label, category in enumerate(self.categories):
            category_path = os.path.join(image_dir, category)
            for image_name in os.listdir(category_path):
                if image_name.endswith(".jpg") or image_name.endswith(".png"):
                    self.images.append(os.path.join(category_path, image_name))
                    self.labels.append(label)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert("RGB")
        
        if self.transform:
            img = self.transform(img)
        
        return img, label, img_path  # بازگرداندن مسیر تصویر همراه با داده‌ها
image_dir = "E:\diginext\proj\good_images"
dataset = CustomDataset(image_dir, transform)
train_data, test_data = train_test_split(dataset, test_size=0.2, stratify=dataset.labels)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)


In [5]:
# آموزش مدل EfficientNet-B0
def train_efficientnet_model(model, train_loader, criterion, optimizer, epochs=EPOCHS):
    model.train()
    for epoch in range(epochs):
        for batch in train_loader:
            # نادیده گرفتن img_path
            images, labels = batch[:2]
            images, labels = images.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_efficientnet_model(efficientnet_model, train_loader, criterion, optimizer)


Epoch 1/30, Loss: 1.672568440437317
Epoch 2/30, Loss: 1.1705032587051392
Epoch 3/30, Loss: 1.2224738597869873
Epoch 4/30, Loss: 0.7761577367782593
Epoch 5/30, Loss: 0.5897956490516663
Epoch 6/30, Loss: 1.127827763557434
Epoch 7/30, Loss: 1.335845947265625
Epoch 8/30, Loss: 1.2812559604644775
Epoch 9/30, Loss: 0.9762377142906189
Epoch 10/30, Loss: 1.7196515798568726
Epoch 11/30, Loss: 0.5134177803993225
Epoch 12/30, Loss: 0.8519454002380371
Epoch 13/30, Loss: 1.1907395124435425
Epoch 14/30, Loss: 0.16143323481082916
Epoch 15/30, Loss: 1.1961842775344849
Epoch 16/30, Loss: 0.8785065412521362
Epoch 17/30, Loss: 1.1057404279708862
Epoch 18/30, Loss: 1.0831271409988403
Epoch 19/30, Loss: 1.1476480960845947
Epoch 20/30, Loss: 0.43111392855644226
Epoch 21/30, Loss: 0.5018068552017212
Epoch 22/30, Loss: 0.7282266020774841
Epoch 23/30, Loss: 0.34304243326187134
Epoch 24/30, Loss: 0.8129590153694153
Epoch 25/30, Loss: 0.7294982075691223
Epoch 26/30, Loss: 0.44478851556777954
Epoch 27/30, Loss: 0

In [109]:
import torch
from sklearn.metrics import accuracy_score

# ارزیابی مدل EfficientNet-B0
def evaluate_efficientnet_model(model, test_loader):
    # انتخاب دستگاه (GPU یا CPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)  # انتقال مدل به دستگاه
    model.eval()  # تنظیم مدل در حالت ارزیابی
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # غیر فعال کردن محاسبات گرادیان برای ارزیابی
        for batch in test_loader:
            # بررسی ساختار داده‌های test_loader
            if len(batch) >= 2:  # اطمینان از وجود حداقل دو مقدار (images و labels)
                images, labels = batch[:2]  # استخراج تصاویر و برچسب‌ها
                images, labels = images.to(device), labels.to(device)  # انتقال داده‌ها به دستگاه
                outputs = model(images)  # پیش‌بینی توسط مدل
                _, preds = torch.max(outputs, 1)  # استخراج کلاس‌های پیش‌بینی شده
                all_preds.extend(preds.cpu().numpy())  # انتقال به CPU و افزودن به لیست پیش‌بینی‌ها
                all_labels.extend(labels.cpu().numpy())  # انتقال به CPU و افزودن به لیست برچسب‌ها
            else:
                print("Invalid batch format in test_loader.")
    
    # محاسبه دقت
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"EfficientNet-B0 Accuracy: {accuracy * 100:.2f}%")

# فراخوانی تابع ارزیابی
evaluate_efficientnet_model(efficientnet_model, test_loader)



EfficientNet-B0 Accuracy: 82.33%


In [ ]:
from sklearn.metrics import classification_report
import torch

# ارزیابی مدل EfficientNet-B0 با نمایش دقت هر دسته
def evaluate_efficientnet_model_per_category(model, test_loader, dataset):
    # انتخاب دستگاه (GPU یا CPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)  # انتقال مدل به دستگاه
    model.eval()  # تنظیم مدل در حالت ارزیابی
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            # استخراج تصاویر و برچسب‌ها (و نادیده‌گرفتن مقادیر اضافی)
            if len(batch) >= 2:
                images, labels = batch[:2]
                images, labels = images.to(device), labels.to(device)
                
                # پیش‌بینی توسط مدل
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                
                # ذخیره نتایج
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            else:
                print("Invalid batch format in test_loader.")
    
    # نمایش دقت کلی
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Overall Accuracy: {accuracy * 100:.2f}%")

    # نمایش دقت هر دسته با استفاده از classification_report
    if hasattr(dataset, "categories"):
        category_names = dataset.categories  # نام دسته‌ها از داده‌ها
    else:
        category_names = [f"Class {i}" for i in range(len(set(all_labels)))]
    
    report = classification_report(all_labels, all_preds, target_names=category_names)
    print("\nCategory-wise Accuracy:\n", report)

# فراخوانی تابع ارزیابی
evaluate_efficientnet_model_per_category(efficientnet_model, test_loader, dataset)


Overall Accuracy: 82.33%

Category-wise Accuracy:
               precision    recall  f1-score   support

         242       0.93      0.84      0.88       236
         248       0.93      0.93      0.93       219
         300       0.91      0.95      0.93       221
         311       0.88      0.94      0.91       170
         321       0.67      0.65      0.66       205
         357       0.90      0.88      0.89       206
         358       0.83      0.93      0.88       206
         365       0.71      0.72      0.71       211
         372       0.66      0.61      0.64       177
         403       0.78      0.76      0.77       215

    accuracy                           0.82      2066
   macro avg       0.82      0.82      0.82      2066
weighted avg       0.82      0.82      0.82      2066



In [132]:
# ذخیره مدل
torch.save(efficientnet_model.state_dict(), 'efficientnet_model.pth')


In [163]:
# چاپ لایه‌های مدل فعلی
#print(efficientnet_model)
# چاپ کلیدهای state_dict ذخیره شده
model = models.efficientnet_b0(pretrained=False)
num_features = efficientnet_model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 10)
model.load_state_dict(torch.load('efficientnet_model.pth', weights_only=True))
print(model.state_dict().keys())
print(efficientnet_model.state_dict().keys())

c:\Users\Latitude 5289 2-in-1\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


odict_keys(['features.0.0.weight', 'features.0.1.weight', 'features.0.1.bias', 'features.0.1.running_mean', 'features.0.1.running_var', 'features.0.1.num_batches_tracked', 'features.1.0.block.0.0.weight', 'features.1.0.block.0.1.weight', 'features.1.0.block.0.1.bias', 'features.1.0.block.0.1.running_mean', 'features.1.0.block.0.1.running_var', 'features.1.0.block.0.1.num_batches_tracked', 'features.1.0.block.1.fc1.weight', 'features.1.0.block.1.fc1.bias', 'features.1.0.block.1.fc2.weight', 'features.1.0.block.1.fc2.bias', 'features.1.0.block.2.0.weight', 'features.1.0.block.2.1.weight', 'features.1.0.block.2.1.bias', 'features.1.0.block.2.1.running_mean', 'features.1.0.block.2.1.running_var', 'features.1.0.block.2.1.num_batches_tracked', 'features.2.0.block.0.0.weight', 'features.2.0.block.0.1.weight', 'features.2.0.block.0.1.bias', 'features.2.0.block.0.1.running_mean', 'features.2.0.block.0.1.running_var', 'features.2.0.block.0.1.num_batches_tracked', 'features.2.0.block.1.0.weight',

In [16]:
import shutil

# تابع ذخیره تصاویر اشتباه
def save_misclassified_images(model, test_loader, output_dir):
    model.eval()
    os.makedirs(output_dir, exist_ok=True)

    with torch.no_grad():
        for images, labels, paths in test_loader:
            images, labels = images.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            probabilities = torch.softmax(outputs, dim=1)
            pred_probs = probabilities[range(len(preds)), preds]

            for img_path, label, pred, prob in zip(paths, labels.cpu().numpy(), preds.cpu().numpy(), pred_probs.cpu().numpy()):
                if label != pred:
                    # فولدر مناسب برای دسته صحیح
                    true_label_dir = os.path.join(output_dir, dataset.categories[label])
                    os.makedirs(true_label_dir, exist_ok=True)

                    # نام فایل بر اساس پیش‌بینی
                    base_name = os.path.basename(img_path)
                    cat_title=code_to_title[code_to_title["new_categoryId"]==int(dataset.categories[pred])]["categoryTitle"].values[0]
                    print(cat_title)
                    print(f"Misclassified: True Label={label}, Predicted={pred}, Probability={prob:.2f}")
                    save_name = f"{prob:.2f}_{cat_title }_{base_name}"
                    save_path = os.path.join(true_label_dir, save_name)

                    shutil.copy(img_path, save_path)
                    print(f"Misclassified image saved: {save_path}")

# فراخوانی تابع
output_dir = "E:/diginext/proj/error_2_images"
save_misclassified_images(efficientnet_model, test_loader, output_dir)


لامپ، چراغ و مهتابی
Misclassified: True Label=5, Predicted=7, Probability=0.36
Misclassified image saved: E:/diginext/proj/error_2_images\357\0.36_لامپ، چراغ و مهتابی_8709141.jpg
میز
Misclassified: True Label=4, Predicted=6, Probability=0.50
Misclassified image saved: E:/diginext/proj/error_2_images\321\0.50_میز_8353760.jpg
ظروف آشپزخانه
Misclassified: True Label=7, Predicted=4, Probability=0.64
Misclassified image saved: E:/diginext/proj/error_2_images\365\0.64_ظروف آشپزخانه_825378.jpg
ظروف آشپزخانه
Misclassified: True Label=0, Predicted=4, Probability=0.51
Misclassified image saved: E:/diginext/proj/error_2_images\242\0.51_ظروف آشپزخانه_8768400.jpg
لوازم خیاطی
Misclassified: True Label=9, Predicted=8, Probability=0.63
Misclassified image saved: E:/diginext/proj/error_2_images\403\0.63_لوازم خیاطی_12017486.jpg
ظروف آشپزخانه
Misclassified: True Label=8, Predicted=4, Probability=0.64
Misclassified image saved: E:/diginext/proj/error_2_images\372\0.64_ظروف آشپزخانه_8430911.jpg
سنگ های قی

In [76]:
code_to_title[code_to_title["new_categoryId"]==300]["categoryTitle"].values[0]

'فرش ماشینی'

In [17]:
import shutil

# تابع ذخیره تصاویر اشتباه
def save_misclassified_images(model, test_loader, output_dir):
    model.eval()
    os.makedirs(output_dir, exist_ok=True)

    with torch.no_grad():
        for images, labels, paths in test_loader:
            images, labels = images.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            probabilities = torch.softmax(outputs, dim=1)
            pred_probs = probabilities[range(len(preds)), preds]

            for img_path, label, pred, prob in zip(paths, labels.cpu().numpy(), preds.cpu().numpy(), pred_probs.cpu().numpy()):
                if label != pred:
                    # فولدر مناسب برای دسته صحیح
                    true_label_dir = os.path.join(output_dir, dataset.categories[label])
                    os.makedirs(true_label_dir, exist_ok=True)

                    # نام فایل بر اساس پیش‌بینی
                    base_name = os.path.basename(img_path)
                    cat_title=code_to_title[code_to_title["new_categoryId"]==int(dataset.categories[pred])]["categoryTitle"].values[0]
                    print(cat_title)
                    print(f"Misclassified: True Label={label}, Predicted={pred}, Probability={prob:.2f}")
                    save_name = f"{prob:.2f}_{cat_title }_{base_name}"
                    save_path = os.path.join(true_label_dir, save_name)

                    shutil.copy(img_path, save_path)
                    print(f"Misclassified image saved: {save_path}")

# فراخوانی تابع
output_dir = "E:/diginext/proj/error_images"
save_misclassified_images(efficientnet_model, train_loader, output_dir)


لامپ، چراغ و مهتابی
Misclassified: True Label=4, Predicted=7, Probability=0.44
Misclassified image saved: E:/diginext/proj/error_images\321\0.44_لامپ، چراغ و مهتابی_8576351.jpg
ظروف آشپزخانه
Misclassified: True Label=7, Predicted=4, Probability=0.44
Misclassified image saved: E:/diginext/proj/error_images\365\0.44_ظروف آشپزخانه_7829612.jpg
ظروف آشپزخانه
Misclassified: True Label=7, Predicted=4, Probability=0.65
Misclassified image saved: E:/diginext/proj/error_images\365\0.65_ظروف آشپزخانه_12445274.jpg
فرش ماشینی
Misclassified: True Label=7, Predicted=2, Probability=0.50
Misclassified image saved: E:/diginext/proj/error_images\365\0.50_فرش ماشینی_11782880.jpg
لوازم خیاطی
Misclassified: True Label=4, Predicted=8, Probability=0.51
Misclassified image saved: E:/diginext/proj/error_images\321\0.51_لوازم خیاطی_3834186.jpg
لامپ، چراغ و مهتابی
Misclassified: True Label=8, Predicted=7, Probability=0.43
Misclassified image saved: E:/diginext/proj/error_images\372\0.43_لامپ، چراغ و مهتابی_341006

In [191]:
import torch
from torchvision import models, transforms
from PIL import Image
import io
import numpy as np
from PIL import Image  # وارد کردن کتابخانه PIL برای باز کردن تصویر
import torch
from torchvision import transforms
import numpy as np
import io
# بارگذاری مدل
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()  # فراخوانی متد __init__ کلاس پایه
        self.model = models.efficientnet_b0(pretrained=True)
        num_features = self.model.classifier[1].in_features
        num_classes = 10  # تعداد گروه‌ها
        self.model.classifier[1] = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

# بارگذاری وزن‌های مدل
model = CustomModel()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)  # انتقال مدل به دستگاه
# بارگذاری state_dict
state_dict = torch.load('efficientnet_model.pth', map_location=device)
print(state_dict.keys())
# حذف پیشوند "model."
state_dict_ = {f"model.{k}": v for k, v in state_dict.items()}

print(state_dict_.keys())
# بارگذاری وزن‌ها در مدل
model.load_state_dict(state_dict_)
#model.load_state_dict(torch.load('efficientnet_model.pth', map_location=device))
model.eval()

# تعریف transform‌ها
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])
# تابع پیش‌بینی تصویر
def predict_image(image_bytes):
    try:
        # استفاده از PIL.Image برای باز کردن تصویر
        img = Image.open(io.BytesIO(image_bytes)).convert('RGB')
        img = transform(img).unsqueeze(0)  # تبدیل به tensor و افزودن dimension batch
        img = img.to(device)

        with torch.no_grad():
            outputs = model(img)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            _,xxx=torch.max(outputs, 1)
        
        # نمایش درصد‌ها
        probabilities = probabilities.cpu().numpy().flatten()
        predicted_class = xxx
        predicted_percentage = probabilities[predicted_class] * 100
        
        # در اینجا ۱۰ گروه برای پیش‌بینی داریم
        groups = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5', 'Group 6', 'Group 7', 'Group 8', 'Group 9', 'Group 10']
        
        return {
            'predicted_class': dataset.categories[predicted_class],
            'predicted_percentage': predicted_percentage,
            'class_probabilities': probabilities.tolist()
        }
    except Exception as e:
        raise ValueError(f"Error processing image: {str(e)}")

# استفاده از پیش‌بینی روی تصویر
image_path = r'app\static\uploads\689392.jpg'  # مسیر تصویر خود را وارد کنید
with open(image_path, 'rb') as f:
    image_bytes = f.read()
    result = predict_image(image_bytes)
    print(result)

odict_keys(['features.0.0.weight', 'features.0.1.weight', 'features.0.1.bias', 'features.0.1.running_mean', 'features.0.1.running_var', 'features.0.1.num_batches_tracked', 'features.1.0.block.0.0.weight', 'features.1.0.block.0.1.weight', 'features.1.0.block.0.1.bias', 'features.1.0.block.0.1.running_mean', 'features.1.0.block.0.1.running_var', 'features.1.0.block.0.1.num_batches_tracked', 'features.1.0.block.1.fc1.weight', 'features.1.0.block.1.fc1.bias', 'features.1.0.block.1.fc2.weight', 'features.1.0.block.1.fc2.bias', 'features.1.0.block.2.0.weight', 'features.1.0.block.2.1.weight', 'features.1.0.block.2.1.bias', 'features.1.0.block.2.1.running_mean', 'features.1.0.block.2.1.running_var', 'features.1.0.block.2.1.num_batches_tracked', 'features.2.0.block.0.0.weight', 'features.2.0.block.0.1.weight', 'features.2.0.block.0.1.bias', 'features.2.0.block.0.1.running_mean', 'features.2.0.block.0.1.running_var', 'features.2.0.block.0.1.num_batches_tracked', 'features.2.0.block.1.0.weight',

C:\Users\Latitude 5289 2-in-1\AppData\Local\Temp\ipykernel_25144\2241605614.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('efficientnet_model.

In [49]:
 for i in range(10):
    print(i,'  ',dataset.categories[i])
    #print("\n")

0    242
1    248
2    300
3    311
4    321
5    357
6    358
7    365
8    372
9    403


In [42]:

with open(image_path, 'rb') as f:
    image_bytes = f.read()
    result = predict_image(image_bytes)
    print(result)

ValueError: Error processing image: type object 'Image' has no attribute 'open'

In [6]:
!pip install transformers

     -------------------------------------- 44.1/44.1 kB 721.1 kB/s eta 0:00:00
     ---------------------------------------- 41.5/41.5 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 10.0/10.0 MB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 447.5/447.5 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 161.8/161.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 274.0/274.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 285.9/285.9 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 2.4/2.4 MB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
from PIL import Image
import torch
import numpy as np


In [25]:

# مسیر مدل ذخیره شده
MODEL_PATH = "vit_model.pth"

# بارگذاری مدل ذخیره شده
def load_model(model_path, num_labels):
    model = ViTForImageClassification.from_pretrained(
        "google/vit-base-patch16-224-in21k",
        num_labels=num_labels
    )
    # بارگذاری مدل با استفاده از map_location برای CPU
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model = model.to("cpu")
    model.eval()
    return model

# بارگذاری مدل
categories = [
            "سنگ‌های قیمتی",
            "مانتو و تونیک",
            "فرش ماشینی",
            "لوازم خیاطی",
            "مبل",
            "کفش و دمپایی زنانه",
            "ظروف آشپزخونه",
            "گیاه خانگی",
            "میز",
            "وسایل الکتریکی",
          

            
        ] # نام گروه‌ها را مشخص کنید
num_labels = len(categories)
model = load_model(MODEL_PATH, num_labels)

# بارگذاری تنظیمات Feature Extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# تابع پیش‌پردازش تصویر
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=img, return_tensors="pt")
    return inputs["pixel_values"].squeeze(0)

# پیش‌بینی احتمال تعلق به هر کلاس
def predict_image(model, image_path, categories):
    image_tensor = preprocess_image(image_path).to("cpu")
    image_tensor = image_tensor.unsqueeze(0)  # افزودن بعد batch
    with torch.no_grad():
        outputs = model(image_tensor).logits
        probabilities = torch.softmax(outputs, dim=1).squeeze(0).cpu().numpy()
    predictions = {categories[i]: float(probabilities[i]) for i in range(len(categories))}
    return predictions


# تست کد با تصویر ورودی
IMAGE_PATH = "good_images/365/1419420.jpg"  # مسیر تصویر ورودی
predictions = predict_image(model, IMAGE_PATH, categories)

# نمایش نتایج
print("Probabilities for each class:")
for class_name, prob in predictions.items():
    print(f"{class_name}: {prob:.4f}")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Latitude 5289 2-in-1\AppData\Local\Temp\ipykernel_15364\2109138393.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by 

Probabilities for each class:
سنگ‌های قیمتی: 0.0001
مانتو و تونیک: 0.0000
فرش ماشینی: 0.0001
لوازم خیاطی: 0.0001
مبل: 0.0000
کفش و دمپایی زنانه: 0.0000
ظروف آشپزخونه: 0.0000
گیاه خانگی: 0.0001
میز: 0.0001
وسایل الکتریکی: 0.9994
